In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'detectron2' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  cu118
detectron2: 0.6


In [4]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.utils.visualizer import ColorMode

In [5]:
os.chdir("/content/drive/MyDrive/Python_HN")

In [6]:
def exec_d2(name,n_classes):
    register_coco_instances(name + "_train", {}, "./coco/coco_annotations_" + name + "_train.json", "./image_dir")
    register_coco_instances(name + "_val", {}, "./coco/coco_annotations_" + name + "_val.json", "./image_dir")
    
    metadata = MetadataCatalog.get(name + "_train")
    dataset_dicts = DatasetCatalog.get(name + "_train")
    
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = (name + "_train",)
    cfg.DATASETS.TEST = ()
    cfg.DATALOADER.NUM_WORKERS = 1
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
    cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
    cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
    if name == "yaelle":
        cfg.SOLVER.MAX_ITER = 1500    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
    else:
        cfg.SOLVER.MAX_ITER = 1000
    cfg.SOLVER.STEPS = []        # do not decay learning rate
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = n_classes  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
    
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    trainer = DefaultTrainer(cfg) 
    trainer.resume_or_load(resume=False)
    trainer.train()
    
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
    predictor = DefaultPredictor(cfg)
    
    evaluator = COCOEvaluator(name + "_val", output_dir="./output")
    val_loader = build_detection_test_loader(cfg, name + "_val")
    f = open(name + '_val_results.txt', 'w+')
    print(inference_on_dataset(predictor.model, val_loader, evaluator), file = f)
    
    return(predictor,metadata)

In [7]:
def classify(name,predictor):
    img_cl = []

    # creates a ScandirIterator aliased as files
    with os.scandir('images') as files:
      # loops through each file in the directory
        for file in files:
            img_cl.append(file.name)
    
    if name == "yaelle":
        count = {}
        for i in img_cl:
            im = cv2.imread('images/' + i)
            outputs = predictor(im)
            hm = 0
            fm = 0
            ef = 0
            nn = 0
            for j in outputs["instances"].pred_classes:
                if j == 0:
                    hm += 1
                elif j == 1:
                    fm += 1
                elif j == 2:
                    ef += 1
                elif j == 3:
                    nn += 1
            nb = str(hm) + '_' + str(fm) + '_' + str(ef) + '_' + str(nn)
            if nb not in count.keys():
                count[nb] = []
                count[nb].append(i)
            else:
                count[nb].append(i)
    else:
        count = {}
        for i in img_cl:
            im = cv2.imread('images/' + i)
            outputs = predictor(im)
            v = Visualizer(im[:, :, ::-1], metadata=metadata, scale=0.5, instance_mode=ColorMode.IMAGE_BW)
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            im_mask = out.get_image()[:, :, ::-1]
            nb = len(outputs["instances"])
            if nb not in count.keys():
                count[nb] = []
                count[nb + '_masks'] = []
                count[nb].append(i)
                
            else:
                count[nb].append(i)
    
    for nb in count.keys():
        nb_path = name + '/' + str(nb)
        os.mkdir(nb_path)
        for filename in count[nb]:
            print(filename)
            file = cv2.imread('images/' + filename)
            cv2.imwrite(nb_path + '/' + filename,file)
        

In [8]:
def classify(name,predictor):
    img_cl = []

    # creates a ScandirIterator aliased as files
    with os.scandir('images') as files:
      # loops through each file in the directory
        for file in files:
            img_cl.append(file.name)
    
    if name == "yaelle":
        count = {}
        for i in img_cl:
            im = cv2.imread('images/' + i)
            outputs = predictor(im)
            v = Visualizer(im[:, :, ::-1], metadata=metadata, scale=0.5, instance_mode=ColorMode.IMAGE_BW)
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            im_mask = out.get_image()[:, :, ::-1]
            hm = 0
            fm = 0
            ef = 0
            nn = 0
            for j in outputs["instances"].pred_classes:
                if j == 0:
                    hm += 1
                elif j == 1:
                    fm += 1
                elif j == 2:
                    ef += 1
                elif j == 3:
                    nn += 1
            nb = str(hm) + '_' + str(fm) + '_' + str(ef) + '_' + str(nn)
            if nb not in count.keys():
                count[nb] = []
                count[nb + '_masks'] = []
                count[nb].append(i)
                count[nb + '_masks'].append(im_mask)
            else:
                count[nb].append(i)
                count[nb + '_masks'].append(im_mask)
    else:
        count = {}
        for i in img_cl:
            im = cv2.imread('images/' + i)
            outputs = predictor(im)
            v = Visualizer(im[:, :, ::-1], metadata=metadata, scale=0.5, instance_mode=ColorMode.IMAGE_BW)
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            im_mask = out.get_image()[:, :, ::-1]
            nb = len(outputs["instances"])
            if str(nb) not in count.keys():
                count[str(nb)] = []
                count[str(nb) + '_masks'] = []
                count[str(nb)].append(i)
                count[str(nb) + '_masks'].append(im_mask)
            else:
                count[str(nb)].append(i)
                count[str(nb) + '_masks'].append(im_mask)
    
    for nb in count.keys():
        if not 'masks' in nb:
            nb_path = name + '/' + str(nb)
            os.mkdir(nb_path)
            for i in range(len(count[nb])):
                file = cv2.imread('images/' + count[nb][i])
                cv2.imwrite(nb_path + '/' + count[nb][i],count[str(nb) + '_masks'][i])
        

In [9]:
names = {}
names['yaelle'] = 4
names['chen'] = 1
names['lise'] = 1
names['ekaterina'] = 1
names['noe'] = 1

In [10]:
for name in names.keys():
    os.chdir("/content/drive/MyDrive/Python_HN")
    predictor,metadata = exec_d2(name,names[name])
    os.chdir('classify_dirs')
    os.mkdir(name)
    classify(name,predictor)

[06/10 17:31:23 d2.data.datasets.coco]: Loaded 100 images in COCO format from ./coco/coco_annotations_noe_train.json
[06/10 17:31:26 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), pa

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[06/10 17:31:26 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[06/10 17:31:40 d2.utils.events]:  eta: 0:08:22  iter: 19  total_loss: 2.458  loss_cls: 0.7948  loss_box_reg: 0.8986  loss_mask: 0.6936  loss_rpn_cls: 0.0356  loss_rpn_loc: 0.02976    time: 0.5346  last_time: 0.3552  data_time: 0.0504  last_data_time: 0.0038   lr: 4.9953e-06  max_mem: 2454M
[06/10 17:31:54 d2.utils.events]:  eta: 0:08:08  iter: 39  total_loss: 2.36  loss_cls: 0.7521  loss_box_reg: 0.8369  loss_mask: 0.6912  loss_rpn_cls: 0.01954  loss_rpn_loc: 0.01399    time: 0.5383  last_time: 0.4352  data_time: 0.0264  last_data_time: 0.0076   lr: 9.9902e-06  max_mem: 2454M
[06/10 17:32:04 d2.utils.events]:  eta: 0:07:50  iter: 59  total_loss: 2.244  loss_cls: 0.6478  loss_box_reg: 0.8067  loss_mask: 0.6839  loss_rpn_cls: 0.04025  loss_rpn_loc: 0.01784    time: 0.5238  last_time: 0.4859  data_time: 0.0193  last_data_time: 0.0192   lr: 1.4985e-05  max_mem: 2454M
[06/10 17:32:13 d2.utils.events]:  eta: 0:07:32  iter: 79  total_loss: 2.194  loss_cls: 0.5495  loss_box_reg: 0.8933  loss_